In [34]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
from collections import deque, defaultdict
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
input_text = au.read_txt_file_lines()
n_rows = len(input_text)
n_cols = len(input_text[0])
for ii in range(1, n_rows):
    assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
print(f'input has {n_rows} rows and {n_cols} cols')

input has 140 rows and 140 cols


In [26]:
dirs = [(1, 0), (0, 1), (-1, 0), (0, -1)]
visited = set()
regions = list()
results = list()

def in_bounds(x, y):
    if x >= 0 and x < n_rows and y >= 0 and y < n_cols:
        return True
    return False

def find_region(x, y):
    seen_perim = set()
    region = [(x, y)]
    visited.add((x, y))
    crop = input_text[x][y]

    ## BFS search":
    queue = deque([(x + d[0], y + d[1], d) for d in dirs])

    while len(queue) > 0:
        (xn,  yn, d_old) = queue.popleft()    
        if not in_bounds(xn, yn):
            seen_perim.add((xn, yn, d_old))  # save old d to account for same border tile from different angles (= different perimeter tiles)
            continue 

        cn = input_text[xn][yn]
        if cn != crop:
            seen_perim.add((xn, yn, d_old))
            continue

        if (xn, yn) in visited:
            continue 

        region.append((xn, yn))
        visited.add((xn, yn))
        for d in dirs:
            queue.append((xn + d[0], yn + d[1], d))

    results.append((len(region), len(seen_perim)))
    regions.append((region, seen_perim))

for i_r, r in enumerate(input_text):
    for i_c, el in enumerate(r):
        if (i_r, i_c) in visited:
            continue

        find_region(i_r, i_c)

# results

In [27]:
sum([r[0] * r[1] for r in results])
# regions

1473408

## Part 2

In [45]:
def perim_converter(perim_set):
    '''Convert existing set of perimeters to new parameter count.
    Idea is: split up by direction. If travelling row-wise (eg d=(1, 0)), then split up by columns of origin. Then, by column, find all jumps in sorted list of rows.
    '''
    dict_perim = {d: [] for d in dirs}
    for (x, y, d) in perim_set:  # split up by direction
        dict_perim[d].append((x, y))

    perim_count = 0 

    for d, tiles in dict_perim.items():
        if d[0] == 0:  # which tile is unique and which one should change. (Eg row-wise means row is unique and cols change)
            ind_unique = 1
            ind_other = 0
        else:
            ind_unique = 0
            ind_other = 1

        dict_tmp = defaultdict(list)
        for t in tiles:  # split up by row/cols of origin
            dict_tmp[t[ind_unique]].append(t[ind_other])

        for _, ind_t_list in dict_tmp.items():
            ind_t_list = sorted(ind_t_list)  # sort
            perim_count += (1 + np.sum(np.diff(ind_t_list) > 1))  # count all jumps. So total is 1 perimeter per row/col of origin, per direction, plus any jumps.
            # print(perim_count, d, _, ind_t_list)
    
    return perim_count

total = 0
for (region, perim_set) in regions:
    new_perim = perim_converter(perim_set)
    total += len(region) * new_perim

total

886364